In [1]:
# default_exp cli

# CLI

> Command line tools for working with storage.

In [2]:
#export
from storage_tools.core import *
from fastcore.script import *

In [3]:
#export
@call_parse
def upload_dataset(storage_name:Param('Section name in config',str),
                   dataset_name:Param('Dataset name',str),
                   config_name:Param('Path to config',str)='secrets/settings.ini',
                   dataset_version:Param('Dataset version',str)='patch'):
    "Create a new dataset archive and upload it to remote storage"
    r=new_storage_client(storage_name,config_name).upload_dataset(dataset_name,dataset_version)
    print('Dataset uploaded to',r)

In [4]:
#export
@call_parse
def download_dataset(storage_name:Param('Section name in config',str),
                     dataset_name:Param('Dataset name',str),
                     config_name:Param('Path to config',str)='secrets/settings.ini',
                     dataset_version:Param('Dataset version',str)='latest',
                     overwrite:Param('If True, delete the dataset and re-download',bool)=False):
    "Download a dataset archive from remote storage"
    r=new_storage_client(storage_name,config_name).download_dataset(dataset_name,dataset_version,overwrite)
    print('Dataset downloaded to',r)

If we have a project folder containing

<pre>
project_root
  &angrt; data
    &angrt; mnist
      &angrt; hand_drawn_digits
        &angrt; digit0.png
        &angrt; digit1.png
        &angrt; ...
  &angrt; secrets
    &angrt; settings.ini
&angrt; main.py
</pre>

where `settings.ini` contains

```
[DEFAULT]
local_path=data

[azure_demo]
storage_client=storage_tools.core.AzureStorageClient
conn_str=<A connection string to an Azure Storage account without credential>
credential=<The credentials with which to authenticate>
container=<The name of a storage container>
```

We can

### Create a new version of a dataset

```
upload_dataset azure_demo mnist/hand_drawn_digits --dataset_version=major
```

### Download the latest version of a dataset

Feel free to delete your local copy of this dataset (from data) to download from azure storage.

```
download_dataset azure_demo mnist/hand_drawn_digits
```

In [5]:
#hide
import shutil
from pathlib import Path
def _rmtree(p):
    try: shutil.rmtree(p)
    except FileNotFoundError: pass

In [6]:
def _make_local_test_data():
    test_files=['a/b/test_data.txt','a/b/more_test_data.txt']
    for i,f in enumerate(test_files):
        f='test/local_path/'+f
        Path(f).parent.mkdir(parents=True,exist_ok=True)
        with open(f, 'w') as _file: _file.write(f'a little bit of data {i}')
    return test_files

In [7]:
for p in ['test/local_path','test/storage_area']: _rmtree(p)

test_files=_make_local_test_data()

def _t(expected,upload_name,version='patch'):
    upload_dataset('local_test',upload_name,'test/settings.ini',version)
_t('a.3.0.0.zip','a','3.0.0')
_t('a.3.0.1.zip','a')
# TODO: check zip contents
_rmtree('test/local_path/a.3.0.0')
_rmtree('test/local_path/a.3.0.1')

download_dataset('local_test','a','test/settings.ini')
download_dataset('local_test','a','test/settings.ini','3.0.0')
download_dataset('local_test','a','test/settings.ini','3.0.0')
download_dataset('local_test','a','test/settings.ini','3.0.0',True)

Dataset uploaded to test\storage_area\a.3.0.0.zip
Dataset uploaded to test\storage_area\a.3.0.1.zip
Dataset downloaded to test\local_path\a.3.0.1
Dataset downloaded to test\local_path\a.3.0.0
Dataset downloaded to test\local_path\a.3.0.0
Dataset downloaded to test\local_path\a.3.0.0


In [8]:
for p in ['test/local_path','test/storage_area']: _rmtree(p)